To create a new project, you can start by defining the scope and objectives of your project. This typically involves identifying the data sources you'll be using, the analyses you plan to perform, and the tools or technologies you'll need. Once you have a clear plan, you can set up a new environment or workspace, such as a new notebook or repository, where you can organize your data, code, and results.

If you need assistance with specific steps or tasks within your project, feel free to ask!

In [1]:
!pip install flask==2.3.3


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


<hr>

generates the answer from the patients with the desired therapist and weekday + slot for the next week

In [2]:
# global variables, should be overwritten
day_global = 'monday'
slot_global = 'morning'
therapist_list_global = ['Dr. Smith']

In [3]:
from datetime import datetime, timedelta, time

def weekday_to_number(day: str) -> int:
    days = {
        'monday': 1,
        'tuesday': 2,
        'wednesday': 3,
        'thursday': 4,
        'friday': 5,
        'saturday': 6,
        'sunday': 7
    }
    return days.get(day.lower(), None)

def slot_to_hour(slot: str) -> int:
    slots = {
        'morning': 8,
        'afternoon': 14
    }
    return slots.get(slot.lower(), None)


def next_week_weekday():
    weekday_num = weekday_to_number(day_global)
    slot_hour = slot_to_hour(slot_global)
    
    if weekday_num is None:
        raise ValueError(f"Invalid weekday: {day_global}")

    today = datetime.now().date()
    today_weekday = today.weekday() + 1
    days_until_next_weekday = (weekday_num - today_weekday + 7) % 7
    if days_until_next_weekday == 0:
        days_until_next_weekday = 7

    next_day = today + timedelta(days=days_until_next_weekday)

    return datetime.combine(next_day, time(hour=slot_hour))


def choose_therapist_from_list():
    global therapist_list_global
    return random.choice(therapist_list_global)


def get_selected_therapist_and_datetime():
    chosen_datetime = next_week_weekday()
    chosen_therapist = choose_therapist_from_list()
    
    return {
        'therapist': chosen_therapist,
        'datetime': chosen_datetime.isoformat() 
    }

Generate an answer from the patient

In [4]:
import random

def weighted_random_answer():
    choices = ['yes', 'no']
    
    # weights must sum up to an arbitrary total (e.g., 10 or 100). --> here 70% yes, 30% no
    weights = [7, 3] 
    result = random.choices(choices, weights=weights, k=1)
    
    return result[0]


Methods used to mock the messaging/e-mail part

In [5]:
def fuse_name(name: str) -> str:
    parts = name.split(' ')
    fused = ''.join(part.replace('.', '') for part in parts)
    return fused.lower()

def create_email_address(name: str) -> str:
    return fuse_name(name) + '@chocolat.com'

def send_fake_email(to, subject, body):
    EMAIL_LOG = []
    EMAIL_LOG.append({
        "to": to,
        "subject": subject,
        "body": body
    })
    print(f"[EMAIL SIMULATED]\n"
      f"to={to}\n"
      f"subject={subject}\n"
      f"body={body}")

In [6]:
def send_therapist_suggestions_list(payload):
    global slot_global, day_global, therapist_list_global
    
    patient = payload['patient']
    therapistList = payload['therapistList']
    slot = payload['slot']
    weekday = payload['weekday']

    patient_address = create_email_address(patient)
    subject = 'Therapist Suggestions'

    send_fake_email(patient_address, subject, therapistList)

    slot_global = slot
    day_global = weekday
    therapist_list_global = therapistList


def send_messages_to_recipients(payload):
    datetime_str = payload['datetime']
    patient = payload['patient']
    therapist = payload['therapist']

    subject = 'Therapy Appointment Details'
    patient_address = create_email_address(patient)
    therapist_address = create_email_address(therapist)

    send_fake_email(patient_address, subject, datetime_str)
    send_fake_email(therapist_address, subject, datetime_str)

Payload Validators

In [7]:
# used to validate the payloads of the POST request

def validate_payload(payload, required_keys):
    if set(payload.keys()) != required_keys:
        return False, 'Payload does not contain necessary fields.'
        
    for key in required_keys:
        if not isinstance(payload[key], str):
            return False, f"Field '{key}' must be a string."

    return True, ""


def validate_therapist_suggestions_payload(payload):
    required_keys = {'therapistList', 'patient', 'weekday', 'slot'}

    if set(payload.keys()) != required_keys:
        return False, 'Payload does not contain necessary fields.'

    if not isinstance(payload['therapistList'], list):
        return False, "Field 'therapistList' must be a list."

    for key in ['patient', 'weekday', 'slot']:
        if not isinstance(payload[key], str):
            return False, f"Field '{key}' must be a string."

    return True, ""


def validate_appointment_msg_payload(payload):
    required_keys = {'datetime', 'therapist', 'patient'}
    return validate_payload(payload, required_keys)

Endpoints

In [8]:
from flask import Flask, jsonify, request
import time as time_module

app = Flask(__name__)
MATCH_MAKER = '/therapist-matcher'
WAITING_TIME = 1


@app.route(MATCH_MAKER + "/therapists-suggestion", methods=["POST"])
def send_therapist_list():
    payload = request.get_json(force=True, silent=True) or {}

    is_valid, error_msg = validate_therapist_suggestions_payload(payload)
    if not is_valid:
        return jsonify({"status": "error", "message": error_msg}), 400
    
    send_therapist_suggestions_list(payload)
    return jsonify({"status": "success", "message": "Message sent successfully."}), 200


@app.route(MATCH_MAKER + "/selection", methods=["GET"])
def handle_selection_request():
    time_module.sleep(WAITING_TIME)
    result = get_selected_therapist_and_datetime()
    print('Chosen therapist and time: ', result)
    return jsonify(result)


@app.route(MATCH_MAKER + "/send-message", methods=["POST"])
def send_message():
    time_module.sleep(WAITING_TIME)
    payload = request.get_json(force=True, silent=True) or {}

    is_valid, error_msg = validate_appointment_msg_payload(payload)
    if not is_valid:
        return jsonify({"status": "error", "message": error_msg}), 400

    send_messages_to_recipients(payload)
    return jsonify({"status": "success", "message": "Message sent successfully."}), 200


@app.route(MATCH_MAKER + "/therapist-response", methods=["GET"])
def get_therapis_response():
    time_module.sleep(WAITING_TIME)
    answer = weighted_random_answer()
    print('Is it a match? ', answer)
    return answer


@app.route(MATCH_MAKER + "/")
def ehr_app():
    return 'The Therapist Matcher Server is running.'


Run it

In [ ]:
# the following line of code will make this notebook act like a server
app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://10.236.143.223:8080
Press CTRL+C to quit
10.236.52.217 - - [12/Dec/2025 04:58:19] "POST /therapist-matcher/therapists-suggestion HTTP/1.1" 200 -
[EMAIL SIMULATED]
to=janastojaodsf@chocolat.com
subject=Therapist Suggestions
body=['Dr. Jordan', 'Dr. Smith']
10.236.52.217 - - [12/Dec/2025 04:58:20] "GET /therapist-matcher/selection HTTP/1.1" 200 -
Chosen therapist and time:  {'therapist': 'Dr. Jordan', 'datetime': '2025-12-17T14:00:00'}
10.236.52.217 - - [12/Dec/2025 04:58:21] "POST /therapist-matcher/send-message HTTP/1.1" 200 -
[EMAIL SIMULATED]
to=janastojaodsf@chocolat.com
subject=Therapy Appointment Details
body=2025-12-17T14:00:00
[EMAIL SIMULATED]
to=drjordan@chocolat.com
subject=Therapy Appointment Details
body=2025-12-17T14:00:00
10.236.52.217 - - [12/Dec/2025 04:58:22] "GET /therapist-matcher/therapist-response HTTP/1.1" 200 -
Is it a 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fbdcce36-fd51-4cfb-8676-e6e544158098' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>